# Введение в обработку текста на естественном языке

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Введение в обработку текста на естественном языке"
* https://www.nltk.org/api/nltk.metrics.distance.html
* https://pymorphy2.readthedocs.io/en/stable/user/guide.html
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [150]:
!pip install pymorphy2

In [151]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Вера\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. При помощи расстояния Левенштейна иправьте опечатку в слове "велечайшим".

In [152]:
from nltk import edit_distance

In [153]:
with open("litw-win.txt", "r", encoding="windows-1251") as fp:
    # делим строки по пробелам и берем последний элемент
    words = [line.split()[-1] for line in fp]

In [154]:
words[10000:10005]

['танцы', 'трубу', 'тулуп', 'турки', 'туфли']

In [155]:
# расстояние Левенштейна измеряет близость между строками; чем меньше полученное значение,
# тем больше похожи две строки друг на друга
# "похожесть" измеряется в количестве операций вставки, удаления и замены символов, которые
# надо сделать, чтобы одну строку свести к другой

edit_distance("Вы справитесь с домашкой", "Вы не справитесь с домашкой")
# вернулось число 3, потому что надо после слова "Вы" добавить 3 символа: 
# пробел, "н" и "е", и тогда две строки станут одинаковыми

3

In [156]:
word = "велечайшим"
# в списке слов words ищем слово, которое как можно ближе к word

# можно воспользоваться встроенным min
min(
    words,  # ищи минимум по строкам из words
    key=lambda w: edit_distance(
        word, w
    ),  # минимум в смысле расстояния от конкретного слова w до слова word
)

'величайшим'

2. Разбейте текст из формулировки второго задания на слова. Проведите стемминг и лемматизацию слов.

In [157]:
from nltk.stem import SnowballStemmer
from nltk import word_tokenize
import pymorphy2

In [158]:
text = '''Разбейте текст из формулировки второго задания на слова. Проведите стемминг и лемматизацию слов в тексте.'''

In [159]:
words = word_tokenize(text)
words

['Разбейте',
 'текст',
 'из',
 'формулировки',
 'второго',
 'задания',
 'на',
 'слова',
 '.',
 'Проведите',
 'стемминг',
 'и',
 'лемматизацию',
 'слов',
 'в',
 'тексте',
 '.']

In [160]:
# создаем специальный объект-стеммер и указываем,
# что будем работать с русским языком (это важно, т.к. другие языки надо обрабатывать по-другому)
stemmer = SnowballStemmer("russian")
# для каждого слова из words вызываем метод stem у стеммера
{w: stemmer.stem(w) for w in words}

{'Разбейте': 'разб',
 'текст': 'текст',
 'из': 'из',
 'формулировки': 'формулировк',
 'второго': 'втор',
 'задания': 'задан',
 'на': 'на',
 'слова': 'слов',
 '.': '.',
 'Проведите': 'провед',
 'стемминг': 'стемминг',
 'и': 'и',
 'лемматизацию': 'лемматизац',
 'слов': 'слов',
 'в': 'в',
 'тексте': 'текст'}

In [161]:
# важно: его надо создать один раз, и использовать ниже по коду
# не создавайте его в циклах и т.д. - это сильно замедлит работу
morph = pymorphy2.MorphAnalyzer()

In [162]:
w = words[0]
# передаем слово в morph.parse и смотрим на результат
res = morph.parse(w)
print(res)
# результат - это список объектов Parse. Каждый объект - это специальная структура, которая
# содержит информацию о том, что это за часть речи, какая у него (слова) нормальная форма
# и некоторую другую информацию
# почему список? потому что часто pymorphy не может однозначно определить, что это за часть речи
# поэтому предлагает нам выбор; этот список упорядочен по убыванию "правдоподобности" результата
# т.е. часто имеет смысл смотреть на 0й элемент
opt1 = res[0]

[Parse(word='разбейте', tag=OpencorporaTag('VERB,perf,tran plur,impr,excl'), normal_form='разбить', score=1.0, methods_stack=((DictionaryAnalyzer(), 'разбейте', 646, 14),))]


In [163]:
# у объектов Parse есть поле normalized, которое вернем нам тоже объект Parse,
# но для нормализованной формы слова
print(opt1.normalized)

# после этого нам останется только вытащить саму строку из объекта
# для этого обратимся к полю word
print(opt1.normalized.word)

Parse(word='разбить', tag=OpencorporaTag('INFN,perf,tran'), normal_form='разбить', score=1.0, methods_stack=((DictionaryAnalyzer(), 'разбить', 646, 0),))
разбить


In [164]:
morph = pymorphy2.MorphAnalyzer()
{w: morph.parse(w)[0].normalized.word for w in words}

{'Разбейте': 'разбить',
 'текст': 'текст',
 'из': 'из',
 'формулировки': 'формулировка',
 'второго': 'второй',
 'задания': 'задание',
 'на': 'на',
 'слова': 'слово',
 '.': '.',
 'Проведите': 'провести',
 'стемминг': 'стемминг',
 'и': 'и',
 'лемматизацию': 'лемматизация',
 'слов': 'слово',
 'в': 'в',
 'тексте': 'текст'}

3. Преобразуйте предложения из формулировки задания 2 в векторы при помощи `CountVectorizer`. Выведите на экран словарь обученного токенизатора.

In [165]:
from sklearn.feature_extraction.text import 

from nltk import sent_tokenize

In [166]:
text = '''Разбейте текст из формулировки второго задания на слова. Проведите стемминг и лемматизацию слов в тексте.'''

In [167]:
# вы знаете, что модели ML любят числа, не любят буквы
# поэтому почти всегда первый наш шаг - преобразовать строку в числовой вектор
# в этом месте появляется куча разнообразных методов, решающих эту задачу
# здесь мы рассмотрим, наверное, самый простой способ решения этой задачи
# на практике так уже не делают, т.к. есть способы кодирования, которые дают более высокие результаты
# но это требует уметь работать с нейронками, поэтому, надеюсь, вам это расскажут на курсе по ML

# мы вместо сложных нейронок сейчас сделаем следующее: возьмем текст, распилим его на слова, запомним, какие слова в нем есть
# каждое слово занумеруем числами от 0 и создадим вектор для каждого предложения по следующему правилу:
# сколько раз в предложении встретилось слово с номером i, такое число будет стоять в i-й координате вектора
# для данного предложения

In [168]:
# можно для этих целей воспользоваться готовым решением из sklearn
# считаю, что с sklearn вы уже знакомы из курса ML

# нам нужно самим разбить текст на предложения
sents = sent_tokenize(text)

# потом обучаем объект-векторизатор (в задании ошибка: не токенизатор, а векторизатор)
# в этом месте происходит разбиение на слова, их нумерация и т.д.
cv = CountVectorizer().fit(sents)
# и потом преобразуем строки в векторы при помощи метода transform обученного векторизатора
# transform вернет sparse массив, сделаем его "обычным", вызвав .toarray()
sents_cv = cv.transform(sents).toarray()
sents_cv

array([[1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]], dtype=int64)

In [169]:
# теперь как понять, что это за единички и нолики?
# посмотрим на поле vocabulary_ у векторизатора
cv.vocabulary_

{'разбейте': 6,
 'текст': 10,
 'из': 2,
 'формулировки': 12,
 'второго': 0,
 'задания': 1,
 'на': 4,
 'слова': 8,
 'проведите': 5,
 'стемминг': 9,
 'лемматизацию': 3,
 'слов': 7,
 'тексте': 11}

In [170]:
# видим, что слову "разбейте" векторизатор присвоил номер 6 (как и почему он это сделал, сейчас не важно)
# смотрим на 6 столбец полученных векторов
sents_cv[:, 6]
# видим, что в 0 строке стоит 1 (это значит, слово "разбейте" встретилось в соответствующем тексте 1 раз)
#  а в 1 строке стоит 0 (этого слова там не было)

array([1, 0], dtype=int64)

In [171]:
# из плюсов: простота реализации, скорость
# из минусов: если слов в словаре много, то векторы становятся очень большие и разреженные 
# и модели начинают плохо работать

In [172]:
# обратите внимание, что есть слово "текст" с номером 10, а есть слово "тексте" с номером 11
# получается, что у нас появится две координаты в векторе, хотя по факту слово-то одно
# это проблема. ее можно решить при помощи стемминга или лемматизации

In [173]:
# в домашке вместо такого алгоритма вы будете использовать TF-IDF; 
# с точки зрения кода это требует от вас взять другой класс из sklearn; 
# на лекции вам подробно расскажут, чем эти подходы различаются

## Лабораторная работа 7


1\. Загрузите данные из файла `ru_recipes_sample.csv` в виде `pd.DataFrame` `recipes` Используя регулярные выражения, удалите из описаний (столбец `description`) все символы, кроме русских букв, цифр и пробелов. Приведите все слова в описании к нижнему регистру. Сохраните полученный результат в столбец `description`.

In [174]:
import re
import random 
import numpy as np
import pandas as pd
import pymorphy2
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.metrics.pairwise import cosine_distances

In [175]:
recipes = pd.read_csv('ru_recipes_sample.csv')

In [176]:
#.str.findall().apply(' '.join)
patt = re.compile(r"[0-9а-яё ]+", re.I)
recipes['description'] = recipes['description'].str.findall(patt).apply(' '.join)
recipes['description'] = recipes['description'].str.lower()
recipes['description'].head().values

array(['этот коктейль готовлю из замороженной клубники  если клубника свежая  то добавляю перепелиное яйцо  благодаря этому коктейль получается устойчиво густым',
       'быстро и вкусно', 'сытный  овощной салатик  пальчики оближете',
       'картофельное пюре и куриные котлеты   вкусная классика в кулинарной книге каждой хозяйки  сегодня я предлагаю вам рецепт нежнейшего картофельного пюре  пропитанного соком котлет  ароматами специй и вкусом зелени вместе с сыром и куриными котлетами',
       'вишневая наливка имеет яркий вишневый вкус  который подчеркивает миндальный аромат косточки и гвоздики  крепость примерно 25  спиртовой тон во вкусе не чувствуется'],
      dtype=object)

2\. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние Левенштейна. Выведите на экран результат в следующем виде:

```
d(word1, word2) = x
```

In [177]:
def make_words(recipe):
    words = []
    words.extend(nltk.word_tokenize(recipe))
    return words

words = sum(list(recipes['description'].apply(make_words)), [])
words

['этот',
 'коктейль',
 'готовлю',
 'из',
 'замороженной',
 'клубники',
 'если',
 'клубника',
 'свежая',
 'то',
 'добавляю',
 'перепелиное',
 'яйцо',
 'благодаря',
 'этому',
 'коктейль',
 'получается',
 'устойчиво',
 'густым',
 'быстро',
 'и',
 'вкусно',
 'сытный',
 'овощной',
 'салатик',
 'пальчики',
 'оближете',
 'картофельное',
 'пюре',
 'и',
 'куриные',
 'котлеты',
 'вкусная',
 'классика',
 'в',
 'кулинарной',
 'книге',
 'каждой',
 'хозяйки',
 'сегодня',
 'я',
 'предлагаю',
 'вам',
 'рецепт',
 'нежнейшего',
 'картофельного',
 'пюре',
 'пропитанного',
 'соком',
 'котлет',
 'ароматами',
 'специй',
 'и',
 'вкусом',
 'зелени',
 'вместе',
 'с',
 'сыром',
 'и',
 'куриными',
 'котлетами',
 'вишневая',
 'наливка',
 'имеет',
 'яркий',
 'вишневый',
 'вкус',
 'который',
 'подчеркивает',
 'миндальный',
 'аромат',
 'косточки',
 'и',
 'гвоздики',
 'крепость',
 'примерно',
 '25',
 'спиртовой',
 'тон',
 'во',
 'вкусе',
 'не',
 'чувствуется',
 'ароматное',
 'тыквенное',
 'суфле',
 'на',
 'песочном',

In [178]:
pairs = [random.sample(words, 2) for pair in range(5)]
pairs

[['приготовлению', 'популярные'],
 ['получилось', 'также'],
 ['и', 'вкусными'],
 ['и', 'любит'],
 ['вкуснее', 'о']]

In [179]:
for pair in pairs:
    print(f'd({pair[0]}, {pair[1]}) = {edit_distance(pair[0],pair[1])}')

d(приготовлению, популярные) = 10
d(получилось, также) = 10
d(и, вкусными) = 7
d(и, любит) = 4
d(вкуснее, о) = 7


3\. Напишите функцию, которая принимает на вход 2 текстовые строки `s1` и `s2` и при помощи расстояния Левенштейна определяет, является ли строка `s2` плагиатом `s1`. Функция должна реализовывать следующую логику: для каждого слова `w1` из `s1` проверяет, есть в `s2` хотя бы одно слово `w2`, такое, что расстояние Левенштейна между `w1` и `w2` меньше 2, и считает количество таких слов в `s1` $P$. 

$$ P = \#\{w_1 \in s_1\ | \exists w_2 \in s_2 : d(w_1, w_2) < tol\}$$

$$ L = max(|s1|, |s2|) $$

Здесь $|\cdot|$ - количество слов в строке, $\#A$ - число элементов в множестве $A$, $w \in s$ означает, что слово $w$ содержится в тексте $s$.

Если отношение $P / L$ больше 0.8, то функция должна вернуть True; иначе False.

Продемонстрируйте работу вашей функции на примере описаний двух рецептов с ID 135488 и 851934 (ID рецепта - это число, стоящее в конце url рецепта). Выведите на экран описания этих рецептов и результат работы функции.

In [180]:
def is_plagiarism(s1: str, s2: str) -> bool:
    p = 0
    for w1 in s1.split():
        for w2 in s2.split():
            if edit_distance(w1,w2) <2:
                p += 1
                break
    l = max(len(s1.split()), len(s2.split()))
    return p/l > 0.8

In [181]:
def last(x):
    return x[-2]

desc1 = recipes['description'][recipes['url'].str.split('/').apply(last) == '135488']
desc2 = recipes['description'][recipes['url'].str.split('/').apply(last) == '851934']
print(desc1.values)
print(desc2.values)

['прекрасной закуской к крепким напиткам на фуршетном столе станет паштет из сала и авокадо с чесноком  который мы спрячем в орешках из бородинского хлеба  положив в середину маслину  закуска получается необычной  оригинальной и ценится особенно мужчинами']
['замечательной закуской к напиткам на фуршетном столе станет паштет из сала и авокадо с чесноком  куда мы добавим орешков из бородинского хлеба  положив в середину маслины  закуска получается крайне необычной  оригинальной и ценится особенно мужчинами']


In [182]:
is_plagiarism(desc1.values[0], desc2.values[0])

True

### Стемминг, лемматизация

4\. На основе набора слов из задания 2 создайте `pd.DataFrame` со столбцами `word`, `stemmed_word` и `normalized_word`. В столбец `stemmed_word` поместите версию слова после проведения процедуры стемминга; в столбец `normalized_word` поместите версию слова после проведения процедуры лемматизации. Столбец `word` укажите в качестве индекса. 

Для стемминга можно воспользоваться `SnowballStemmer` из `nltk`, для лемматизации слов - пакетом `pymorphy2`. Сравните результаты стемминга и лемматизации. Поясните на примере одной из строк получившегося фрейма (в виде текстового комментария), в чем разница между двумя этими подходами. 

In [183]:
words = np.array(pairs).reshape(-1,1)
d = {'word':words[:,0]}
words_df = pd.DataFrame(d)

In [184]:
stemmer = SnowballStemmer("russian")
words_df['stemmed_word'] = words_df['word'].apply(lambda x: stemmer.stem(x))

morph = pymorphy2.MorphAnalyzer()
words_df['normalized_word'] = words_df['word'].apply(lambda x: morph.parse(x)[0].normalized.word)
words_df

,word,stemmed_word,normalized_word
0,приготовлению,приготовлен,приготовление
1,популярные,популярн,популярный
2,получилось,получ,получиться
3,также,такж,также
4,и,и,и
5,вкусными,вкусн,вкусный
6,и,и,и
7,любит,люб,любить
8,вкуснее,вкусн,вкусный
9,о,о,о


In [185]:
words_df.set_index('word')

,stemmed_word,normalized_word
word,,
приготовлению,приготовлен,приготовление
популярные,популярн,популярный
получилось,получ,получиться
также,такж,также
и,и,и
вкусными,вкусн,вкусный
и,и,и
любит,люб,любить
вкуснее,вкусн,вкусный


Стемминг и лемматизация - два подхода, которые помогают программе воспринимать разные формы слова как одно во время анализа данных.
Суть стемминга заключается в отбрасывании последних нескольких символов. Например, приготовлению -> приготовлен
А с помощью лемматизации слово приводится к начальной форме, по правилам русского языка. Например, приготовлению -> приготовление

5\. Добавьте в таблицу `recipes` столбец `description_no_stopwords`, в котором содержится текст описания рецепта после удаления из него стоп-слов. Посчитайте и выведите на экран долю стоп-слов среди общего количества слов. Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [186]:
stop_words = stopwords.words('russian')

In [187]:
def stopwordsdel(desc):
    desc1 = ''
    for word in desc.split(' '):
        if (word in stop_words) == False:
            desc1 = desc1 + ' ' + word   
    return desc1
    
recipes['description_no_stopwords'] = recipes['description'].apply(stopwordsdel)

In [188]:
words_number = recipes['description'].apply(len).sum()

In [189]:
stop_words_number = recipes['description_no_stopwords'].apply(len).sum()

In [190]:
f'{100*(words_number - stop_words_number)/words_number}%'

'15.685607349438584%'

In [191]:
def word_func(desc):
    tokenizer = nltk.tokenize.RegexpTokenizer((r"[а-яёА-ЯЁ]+"))
    return tokenizer.tokenize(desc)

morph = pymorphy2.MorphAnalyzer()

def morph_func(desc):
    words_return = []
    for s in desc:
        words_return.append(morph.parse(s)[0].normalized.word)
    return words_return

all_words = recipes['description'].apply(word_func).apply(morph_func).sum()

In [192]:
unique_words, unique_counts = np.unique(np.array(all_words), return_counts=True)
unique_dict = dict(zip(unique_words, unique_counts))
list(dict(sorted(unique_dict.items(), key=lambda x: x[1], reverse = True)).keys())[:10]

['и', 'в', 'с', 'на', 'очень', 'не', 'я', 'рецепт', 'вкусный', 'это']

In [193]:
all_nostopwords = recipes['description_no_stopwords'].apply(word_func).apply(morph_func).sum()

In [194]:
unique_words, unique_counts = np.unique(np.array(all_nostopwords), return_counts=True)
unique_dict = dict(zip(unique_words, unique_counts))
list(dict(sorted(unique_dict.items(), key=lambda x: x[1], reverse = True)).keys())[:10]

['очень',
 'рецепт',
 'вкусный',
 'это',
 'блюдо',
 'вкус',
 'нежный',
 'приготовить',
 'салат',
 'приготовление']

### Векторное представление текста

6\. Выберите случайным образом 5 рецептов из набора данных, в названии которых есть слово "оладьи" (без учета регистра). Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`. На основе полученных векторов создайте `pd.DataFrame`, в котором названия колонок соответствуют словам из словаря объекта-векторизатора. 

Примечание: обратите внимание на порядок слов при создании колонок.

In [195]:
def name_func(name):
    tokenizer = nltk.tokenize.RegexpTokenizer((r"[а-яёА-ЯЁ]+"))
    return tokenizer.tokenize(name.lower())

desc_sample = recipes[recipes['name'].apply(name_func).apply(lambda x: "оладьи" in x)].sample(5)
desc_sample

,url,name,ingredients,description,description_no_stopwords
135,https://www.povarenok.ru/recipes/show/149119/,Кокосовые оладьи,"{'Сыворотка': '200 мл', 'Мука пшеничная': '200...",порадуйте себя и близких пышными оладьями с ко...,порадуйте близких пышными оладьями кокосовым ...
926,https://www.povarenok.ru/recipes/show/40334/,Оладьи из капусты с брынзой,"{'Кефир': '200 мл', 'Крупа манная': '2 ст. л.'...",нежные оладушки которые просто тают во рту,нежные оладушки которые просто тают рту
614,https://www.povarenok.ru/recipes/show/103747/,Тыквенные оладьи с лимоном,"{'Тыква': '300 г', 'Цедра лимона': '1 ч. л.', ...",нежные сладкие оладушки с лимонной ноткой н...,нежные сладкие оладушки лимонной ноткой ск...
3390,https://www.povarenok.ru/recipes/show/103306/,"Оладьи из сельдерея, кабачков, феты и чечевицы","{'Сельдерей черешковый': '1 пуч.', 'Кабачок': ...",такие оладьи любят в греции и турции в качеств...,такие оладьи любят греции турции качестве зак...
2122,https://www.povarenok.ru/recipes/show/138548/,Голландские лимонные оладьи,"{'Молоко': '4 ст. л.', 'Дрожжи': '6 г', 'Вода'...",голландские оладьи особенность которых зак...,голландские оладьи особенность которых за...


In [196]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tfidf_desc = vectorizer.fit(desc_sample['description'].values)

In [197]:
desc_array = tfidf_desc.transform(desc_sample['description'].values).toarray()

In [198]:
dict_desc = tfidf_desc.vocabulary_

In [199]:
tfidf_df = pd.DataFrame(data = desc_array, columns = dict(sorted(dict_desc.items(), key=lambda item: item[1])).keys())
tfidf_df

,белки,близких,блюда,более,быстро,варенье,вкус,вкусом,во,голландские,...,форме,хороши,чем,черешки,чечевица,что,чуть,шарообразной,шоколадный,этом
0,0.000000,0.306413,0.000000,0.000000,0.000000,0.000000,0.247212,0.306413,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.409865,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.169996,0.000000,0.169996,0.000000,0.000000,0.000000,0.137152,0.000000,0.000000,0.000000,...,0.000000,0.169996,0.000000,0.169996,0.169996,0.000000,0.000000,0.000000,0.000000,0.169996
4,0.000000,0.000000,0.000000,0.114396,0.114396,0.114396,0.000000,0.000000,0.000000,0.228792,...,0.114396,0.000000,0.114396,0.000000,0.000000,0.114396,0.114396,0.114396,0.114396,0.000000


7\. Вычислите близость между каждой парой рецептов, выбранных в задании 6, используя косинусное расстояние (можно воспользоваться функциями из любого пакета: `scipy`, `scikit-learn` или реализовать функцию самому). Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

Примечание: обратите внимание, что $d_{cosine}(x, x) = 0$

In [200]:
from scipy.spatial import distance

cosine_arr = []
for i in range(len(desc_array)):
    cosine_arr.append([])
    for j in range(len(desc_array)):
        cosine_arr[i].append(distance.cosine(desc_array[i], desc_array[j]))   
        
cosine_arr

[[0.0, 1.0, 1.0, 0.9660944749814976, 1.0],
 [1.0, 0.0, 0.7976431145313398, 1.0, 0.969480465316354],
 [1.0, 0.7976431145313398, 0.0, 1.0, 0.9152812248237238],
 [0.9660944749814976, 1.0, 1.0, 0.0, 0.9493667724711841],
 [1.0, 0.969480465316354, 0.9152812248237238, 0.9493667724711841, 0.0]]

In [201]:
cos_df = pd.DataFrame(data=cosine_arr, index=desc_sample['name'].values, columns=desc_sample['name'].values)
cos_df

,Кокосовые оладьи,Оладьи из капусты с брынзой,Тыквенные оладьи с лимоном,"Оладьи из сельдерея, кабачков, феты и чечевицы",Голландские лимонные оладьи
Кокосовые оладьи,0.000000,1.000000,1.000000,0.966094,1.000000
Оладьи из капусты с брынзой,1.000000,0.000000,0.797643,1.000000,0.969480
Тыквенные оладьи с лимоном,1.000000,0.797643,0.000000,1.000000,0.915281
"Оладьи из сельдерея, кабачков, феты и чечевицы",0.966094,1.000000,1.000000,0.000000,0.949367
Голландские лимонные оладьи,1.000000,0.969480,0.915281,0.949367,0.000000


8\. Напишите функцию, которая принимает на вход `pd.DataFrame`, полученный в задании 7, и возвращает в виде кортежа названия двух различных рецептов, которые являются наиболее похожими. Прокомментируйте результат (в виде текстового комментария). Для объяснения результата сравните слова в описаниях двух этих отзывов.

In [202]:
def find_closest(cos_df) -> tuple:
    return cos_df[cos_df != 0].stack().idxmin()

In [203]:
cos_df[cos_df != 0]

,Кокосовые оладьи,Оладьи из капусты с брынзой,Тыквенные оладьи с лимоном,"Оладьи из сельдерея, кабачков, феты и чечевицы",Голландские лимонные оладьи
Кокосовые оладьи,NaN,1.000000,1.000000,0.966094,1.000000
Оладьи из капусты с брынзой,1.000000,NaN,0.797643,1.000000,0.969480
Тыквенные оладьи с лимоном,1.000000,0.797643,NaN,1.000000,0.915281
"Оладьи из сельдерея, кабачков, феты и чечевицы",0.966094,1.000000,1.000000,NaN,0.949367
Голландские лимонные оладьи,1.000000,0.969480,0.915281,0.949367,NaN


In [204]:
find_closest(cos_df)

('Оладьи из капусты с брынзой', 'Тыквенные оладьи с лимоном')

In [205]:
closest_desc = desc_sample[desc_sample['name'].isin(find_closest(cos_df))]['description']

In [206]:
print(closest_desc.iloc[0])
print('')
print(closest_desc.iloc[1])

нежные оладушки  которые просто тают во рту

нежные  сладкие оладушки с лимонной ноткой   на скорую руку


Могу предположить, что данные описания имеют наименьшее расстояние из-за того, что оба содержат словосочетание "нежные оладушки"